In [1]:
!pip install --upgrade langchain
!pip install gradio
!pip install -U langchain-community
!pip install --upgrade langchain-openai
!pip install transformers
!pip install --upgrade langchain-core
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing 

In [2]:
import os
import glob
import gradio as gr
from google.colab import userdata
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from enum import Enum
from langchain.vectorstores import FAISS
from pathlib import Path
import torch


In [3]:
class Rag(Enum):
  MODEL = 'gpt-4o-mini'
  EMBED_MODEL = 'text-embedding-ada-002'


In [4]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
def add_metadata(document,doc_type):

  document.metadata['doc_type'] = doc_type

  return document

In [6]:
def process_file(file_path):
    """
    Process a single markdown file

    Args:
        file_path: Path to the markdown file

    Returns:
        List of document chunks with metadata
    """
    try:

        doc_type = os.path.basename(file_path).replace('.md', '')

        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        doc = Document(page_content=content, metadata={'source': file_path})

        doc = add_metadata(doc, doc_type)

        text_splitter = CharacterTextSplitter(chunk_size=1000)
        chunks = text_splitter.split_documents([doc])

        print(f"Processed {len(chunks)} chunks from {file_path}")

        return chunks

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return []

In [7]:
def create_vector_store(chunks):
    # Check if chunks list is empty
    if not chunks:
        raise ValueError("No document chunks to process. Please check your knowledge base directory.")

    # Use OpenAI Embeddings for better quality
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

    # Create vector store using FAISS
    vector_store = FAISS.from_documents(chunks, embeddings)

    return vector_store


In [8]:
def get_vectorstore_info(vector_store):
    try:
        # Get the number of vectors in the vector store
        num_vectors = len(vector_store.index_to_docstore_id)

        # Basic vector store info
        vectorstore_info = {
            "Number of Vectors": num_vectors,
            "Store Type": type(vector_store).__name__
        }

        return vectorstore_info
    except Exception as e:
        return {"Error": str(e)}


In [10]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(
        temperature=0.7,
        model_name=Rag.MODEL.value,
        api_key=OPENAI_API_KEY
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
        output_key='answer'
    )

    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Retrieve 5 most relevant documents
    )

    conversational_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        verbose=True
    )

    return conversational_chain


In [11]:
def chat(message, history):
    # Format previous history for the chain
    response = conversational_chain({"question": message})

    # Add source information
    sources = []
    for doc in response.get("source_documents", []):
        if "doc_type" in doc.metadata:
            sources.append(doc.metadata["doc_type"])

    answer = response["answer"]

    # Optionally add sources to the response
    if sources:
        unique_sources = list(set(sources))
        answer += f"\n\nSources: {', '.join(unique_sources)}"

    return answer


In [12]:
def format_chat_history(history):
    formatted_history = []
    for human_msg, ai_msg in history:
        formatted_history.append(f"Human: {human_msg}")
        formatted_history.append(f"Assistant: {ai_msg}")
    return formatted_history


In [13]:
def main():
    knowledge_base_path = '/content/drive/MyDrive/Knowledge_base'

    # Check if the knowledge base directory exists
    if not os.path.exists(knowledge_base_path):
        print(f"Knowledge base directory not found: {knowledge_base_path}")
        return

    # Find all .md files directly in the knowledge base directory
    md_files = list(Path(knowledge_base_path).glob('*.md'))

    if not md_files:
        print(f"No .md files found in {knowledge_base_path}")
        return

    print(f"Found {len(md_files)} markdown files in knowledge base:")
    for file in md_files:
        print(f"  - {file}")

    # Process all markdown files
    chunks = []
    for file in md_files:
        file_chunks = process_file(str(file))
        chunks.extend(file_chunks)

    # Check if any chunks were created
    if not chunks:
        print("No document chunks were created. Check your knowledge base content.")
        return

    print(f"Total chunks created: {len(chunks)}")

    # Create vector store from chunks
    vector_store = create_vector_store(chunks)

    # Get vector store info
    vectorstore_info = get_vectorstore_info(vector_store)
    print("Vector Store Information:")
    for key, value in vectorstore_info.items():
        print(f"  {key}: {value}")

    # Create conversation chain
    global conversational_chain
    conversational_chain = get_conversation_chain(vector_store)

    # Create Gradio interface with a nicer UI
    with gr.Blocks(title="Personal Knowledge Assistant") as demo:
        gr.Markdown("# Your Personal Knowledge Assistant")
        gr.Markdown("Ask questions about your personal information and knowledge base")

        chatbot = gr.ChatInterface(
            chat,
            chatbot=gr.Chatbot(height=600),
            title="Personal Knowledge Assistant",
            description="Your personal AI assistant with access to your knowledge base"
        )

    # Launch the interface
    demo.launch(inbrowser=True)


In [14]:
if __name__ == "__main__":
    main()

Found 12 markdown files in knowledge base:
  - /content/drive/MyDrive/Knowledge_base/Personal Information.md
  - /content/drive/MyDrive/Knowledge_base/Academic Profile.md
  - /content/drive/MyDrive/Knowledge_base/Major Projects.md
  - /content/drive/MyDrive/Knowledge_base/TODOs.md
  - /content/drive/MyDrive/Knowledge_base/Friends & University Network.md
  - /content/drive/MyDrive/Knowledge_base/AI Society Two-Year Roadmap.md
  - /content/drive/MyDrive/Knowledge_base/Daily Routine.md
  - /content/drive/MyDrive/Knowledge_base/Likes & Dislikes.md
  - /content/drive/MyDrive/Knowledge_base/Goals & Dreams.md
  - /content/drive/MyDrive/Knowledge_base/RAG-based Knowledge Agent Design.md
  - /content/drive/MyDrive/Knowledge_base/Favorite Motivational Quotes.md
  - /content/drive/MyDrive/Knowledge_base/Personal Reflections.md
Processed 1 chunks from /content/drive/MyDrive/Knowledge_base/Personal Information.md
Processed 1 chunks from /content/drive/MyDrive/Knowledge_base/Academic Profile.md
Proc

<ipython-input-10-623ea894b9e7>:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
<ipython-input-13-ac76f980c66b>:53: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=600),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e844f31927a9a01525.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
